In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os

import pandas as pd
import seaborn as sn
import torch
from IPython.core.display import display
!pip install pytorch_lightning
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

In [3]:
class MNISTModel(LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = torch.nn.Linear(28*28, 10)

  def forward(self, x):
    return torch.relu(self.l1(x.view(x.size(0), -1)))

  def training_step(self, batch, batch_idx):
    x, y = batch
    loss = F.cross_entropy(self(x), y)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=0.02)

In [4]:
# init our model
mnist_model = MNISTModel()

# init DataLoader from  MNIST dataset
train_ds = MNIST(PATH_DATASETS, train=True, download=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

# initialize a trainer
trainer = Trainer(
    accelerator="auto",
    devices = 1 if torch.cuda.is_available() else None,
    max_epochs = 5,
    callbacks = [TQDMProgressBar(refresh_rate=20)],
)

# train the model
trainer.fit(mnist_model, train_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]